# Setup

The main focus of this notebook is to analyze and mitigate potential biases in a dataset. The notebook uses various techniques and technologies, such as [AIF360](https://aif360.mybluemix.net), [Fairlearn](https://fairlearn.org) , and [scikit-learn](https://scikit-learn.org/stable/), to preprocess and analyze the data, and to train and evaluate machine learning models. The notebook also includes visualizations and statistics to help understand the distribution and correlations of the data, and to identify any potential biases.

**PLEASE NOTE**: The notebook must be configured with a dataset and some configuration variables.  
The ***protected attributes*** must be categorical and binary (0,1), but the original column must be mantained as a continuous variable.

## Colab
In order to run this notebook in Google Colab, you have to:
1. Upload the project folder to your Google Drive
2. Mount your Google Drive in the next code cell
3. Update the `path_to_project` variable, with the path to the project folder in your Google Drive (e.g. `path_to_project = '/content/drive/MyDrive/Colab/project'`), this can be seen from the file tab on the left of the Colab interface
4. Update the pip installation command  (e.g. `!pip install -r /content/drive/MyDrive/Colab/project/requirements.txt`)

In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  import sys
  path_to_project = '/content/drive/MyDrive/FairAlgorithm'  #UPDATE THIS LINE
  sys.path.append(path_to_project)
  !sudo apt install libcairo2-dev pkg-config python3-dev
  %pip install -r /content/drive/MyDrive/FairAlgorithm/source/requirements.txt  #UPDATE THIS LINE
  IN_COLAB = True
except:
  IN_COLAB = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcairo2-dev is already the newest version (1.16.0-5ubuntu2).
pkg-config is already the newest version (0.29.2-1ubuntu3).
python3-dev is already the newest version (3.10.6-1~22.04.1).
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'sudo apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.


In [4]:
from source.utils.config import *
from source.utils.mitigation_utils import *

pip install 'aif360[inFairness]'


In [5]:
!pip install BlackBoxAuditing

## Configure the notebook


In the next code cell, set all the variables that are used throughout the notebook.  
The variables are used to configure the notebook, and to set the paths to the data files.

Modify the variables in the next code cell to configure the notebook

- `dataset_name`: The name of the dataset file.
- `sensible_attribute`: The sensible attribute to use for bias mitigation.

In [6]:
#INPUT
dataset_name = "diabetes-prediction"

In [7]:
dataset_config = datasets_config[dataset_name]
ignore_cols = dataset_config['ignore_cols']
target_variable = dataset_config['target_variable']
target_variable_labels = dataset_config['target_variable_labels']
sensible_attributes = dataset_config['sensible_attributes']
sensible_attribute = sensible_attributes[0]
default_mappings = dataset_config.get('default_mappings', {})
n_splits = dataset_config['n_splits']

privileged_groups = aif_config[dataset_name][sensible_attribute]['privileged_groups']
unprivileged_groups = aif_config[dataset_name][sensible_attribute]['unprivileged_groups']
reduced_df_techniques = aif_config[dataset_name][sensible_attribute]['reduced_df_techniques']
params = aif_config[dataset_name][sensible_attribute]['params']

dataset_path = path_to_project + '/data/preprocessed/preprocessed-{}.csv'.format(dataset_name) if IN_COLAB else 'data/preprocessed/preprocessed-{}.csv'.format(dataset_name)
df = pd.read_csv(dataset_path)
df = df.drop(columns=ignore_cols)
feature_cols = df.columns

In [8]:
config = {}
config['df']= df
config['dataset_name'] = dataset_name
config['target_variable'] = target_variable
config['sensible_attribute'] = sensible_attribute
config['path_to_project'] = path_to_project
config['n_splits'] = n_splits
config['models'] = models
config['random_seed'] = random_seed
config['privileged_groups'] = privileged_groups
config['unprivileged_groups'] = unprivileged_groups
config['default_mappings'] = default_mappings
config['reduced_df_techniques'] = reduced_df_techniques
config['params'] = params

## Load the data
Data is loaded from the file specified in the `dataset_path` variable using the `pandas` library.  
[pandas](https://pandas.pydata.org) is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language.

In [9]:
df.head(5)

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_category,gender_category,...,glyburide-metformin_category,glipizide-metformin_category,glimepiride-pioglitazone_category,metformin-rosiglitazone_category,metformin-pioglitazone_category,age_mean,age_category,change_category,diabetes-medication_category,diabetes
0,1,41,0,1,0,0,0,1,1,0,...,0,0,0,0,0,5,0,0,0,0
1,3,59,0,18,0,0,0,9,1,0,...,0,0,0,0,0,15,0,1,1,1
2,2,44,1,16,0,0,0,7,1,1,...,0,0,0,0,0,35,1,1,1,0
3,1,51,0,8,0,0,0,5,1,1,...,0,0,0,0,0,45,1,1,1,0
4,3,31,6,16,0,0,0,9,1,1,...,0,0,0,0,0,55,1,0,1,1


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73450 entries, 0 to 73449
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   time_in_hospital                   73450 non-null  int64
 1   num_lab_procedures                 73450 non-null  int64
 2   num_procedures                     73450 non-null  int64
 3   num_medications                    73450 non-null  int64
 4   number_outpatient                  73450 non-null  int64
 5   number_emergency                   73450 non-null  int64
 6   number_inpatient                   73450 non-null  int64
 7   number_diagnoses                   73450 non-null  int64
 8   race_category                      73450 non-null  int64
 9   gender_category                    73450 non-null  int64
 10  metformin_category                 73450 non-null  int64
 11  repaglinide_category               73450 non-null  int64
 12  nateglinide_catego

## Mitigation utils


In [ ]:
def train_test_splitting(df, n_splits):
  df_splitting = {}

  w = int(len(df)/n_splits)
  window = w
  start_point = 0
  for i in range(0,n_splits):
      train = {}
      test = {}
      df_train_1 = {}
      df_train_2 = {}
      df_test = df[start_point:window]
      if i != 0:
        df_train_1 = df[0: start_point]

      if i != n_splits-1:
        df_train_2 = df[window: len(df)]

      if (i != 0 and  i != n_splits-1):
        concat_df = [df_train_1, df_train_2]
        df_train = pd.concat(concat_df)
      elif i != 0:
        df_train = df_train_1
      else:
        df_train = df_train_2

      start_point= window
      window = window + w

      df_splitting[i] = {'train': df_train, 'test': df_test}
  return df_splitting

In [ ]:
def df_X_Y_split(df_train, df_test, target_variable):
  Y_train = df_train[target_variable]
  X_train = df_train.drop(target_variable, axis=1)
  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  return X_train, Y_train, X_test, Y_test

In [ ]:
def df_X_Y_split_2D(df_train, df_test, target_variable, sensible_attribute):
  Y_train = df_train[target_variable].values
  S_train = df_train[sensible_attribute].values
  X_train = df_train.drop(target_variable, axis=1)
  X_train = X_train.drop(sensible_attribute, axis=1)

  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  S_test = df_test[sensible_attribute].values

  X_train = normalize(X_train)
  X_test = normalize(X_test)

  pca = PCA(n_components=2)
  X_train2D = pca.fit(X_train).transform(X_train)
  X_test2D = pca.fit(X_test).transform(X_test)
  return X_train2D, Y_train, X_test2D, Y_test, S_train, S_test

In [ ]:
def df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute):
  Y_train = df_train[target_variable].values
  S_train = df_train[sensible_attribute].values
  X_train = df_train.drop(target_variable, axis=1)

  Y_test = df_test[target_variable]
  X_test = df_test.drop(target_variable, axis=1)
  S_test = df_test[sensible_attribute].values

  return X_train, Y_train, X_test, Y_test, S_train, S_test

In [ ]:
def compute_predictions_and_tests(df, target_variable, n_splits, models, n_estimators, random_seed):
  predicted_and_real_values = {}
  for model_name in tqdm(models):
    clf = models[model_name]
    df_splitting = train_test_splitting(df, n_splits)
    pred_and_y = {}
    for i in range(0,n_splits):
      df_split = df_splitting[i]
      df_train = df_split['train']
      df_test = df_split['test']

      X_train, Y_train, X_test, Y_test = df_X_Y_split(df_train, df_test, target_variable)
      clf.fit(X_train,Y_train)
      y_pred = clf.predict(X_test)

      S_test = X_test[sensible_attribute].values
      pred_and_y[i] = {'y_test': Y_test.to_numpy().astype(int), 'y_pred': y_pred.astype(int), 's_test':  S_test.astype(int)}

    predicted_and_real_values[model_name] = pred_and_y

  return predicted_and_real_values


In [ ]:
def fl_am_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, classifier):
  predicted_and_real_values = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']
    ##Input data should be in 2-dimensions
    X_train2D, Y_train, X_test2D, Y_test, S_train, S_test = df_X_Y_split_2D(df_train, df_test, target_variable, sensible_attribute)
    #print(len(X_train2D), len(Y_train),len(S_train))

    #classifier is given in in-processing mitigation
    predictor = classifier.fit(X_train2D, Y_train, sensitive_features=S_train)
    y_pred = predictor.predict(X_test2D)
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred, 's_test':  S_test }

  predicted_and_real_values = pred_and_y

  return predicted_and_real_values

In [ ]:
def fl_to_compute_predictions_and_tests(df, target_variable, sensible_attribute, n_splits, classifier):
  predicted_and_real_values = {}

  df_splitting = train_test_splitting(df, n_splits)
  pred_and_y = {}
  for i in range(0,n_splits):

    df_split = df_splitting[i]
    df_train = df_split['train']
    df_test = df_split['test']
    ##Input data should be in 2-dimensions
    X_train, Y_train, X_test, Y_test, S_train, S_test = df_X_Y_split_S(df_train, df_test, target_variable, sensible_attribute)
    #print(len(X_train2D), len(Y_train),len(S_train))

    #classifier is given in in-processing mitigation
    predictor = classifier.fit(X_train, Y_train, sensitive_features=S_train)
    y_pred = predictor.predict(X_test, sensitive_features=S_test)
    pred_and_y[i] = {'y_test': Y_test.to_numpy(), 'y_pred': y_pred, 's_test':  S_test}

  predicted_and_real_values = pred_and_y

  return predicted_and_real_values

In [ ]:
def save_predictions_and_tests(predicted_and_test, dataset_name, sensible_attribute, mitigation,path_to_project):
  save_path = path_to_project + '/data/predictions_and_tests/pred_test-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
  with open(save_path, 'wb') as fp:
    pickle.dump(predicted_and_test, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def save_mitigated_dataset(mitigated_dataset, path_to_project,dataset_name, sensible_attribute, mitigation):
  mitigated_dataset.to_csv("{}/data/mitigated/mitigated-{}-{}-{}.csv".format(path_to_project,dataset_name, sensible_attribute, mitigation), sep=',', index=False, encoding='utf-8')

In [ ]:
def check_results(results):
  for model_name in tqdm(models):
    for i in range(0,n_splits):
      y_test = results[model_name][i]['y_test']
      y_pred = results[model_name][i]['y_pred']
      for j in range(0,67):
        if y_test[j] != y_pred[j]:
          print(y_test[j], y_pred[j])

# Fairlearn
[Fairlearn](https://fairlearn.org) is an open source toolkit that empowers developers of artificial intelligence (AI) systems to assess their system's fairness and mitigate any observed unfairness issues.

### Correlation Remover
Pre-processing: mitigated dataset changes

MITIGATED DATASET

ALPHA PARAMETER SET TO DEFAULT

NO MODEL ENFORCEMENT

In [11]:
print(fl_cr(config))

  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

(
           time_in_hospital  num_lab_procedures  num_procedures  num_medications  \
0              1.021027           41.328073       -0.036802         0.756052   
1              3.021027           59.328073       -0.036802        17.756052   
2              2.021027           44.328073        0.963198        15.756052   
3              1.021027           51.328073       -0.036802         7.756052   
4              3.021027           31.328073        5.963198        15.756052   
...                 ...                 ...             ...              ...   
73445          5.021027           21.328073        0.963198        15.756052   
73446          2.934578           49.979236        0.114506        16.759017   
73447          4.934578           31.979236        3.114506        18.759017   
73448         10.021027           45.328073        1.963198        20.756052   
73449          6.021027           13.328073        2.963198         2.756052   

       number_outpatient  number_emergency  number_inpatient  \
0                    0.0               0.0          0.004734   
1                    0.0               0.0          0.004734   
2                    0.0               0.0          0.004734   
3                    0.0               0.0          0.004734   
4                    0.0               0.0          0.004734   
...                  ...               ...               ...   
73445                0.0               0.0          1.004734   
73446                0.0               0.0         -0.014728   
73447                0.0               0.0          0.985272   
73448                0.0               0.0          1.004734   
73449                0.0               0.0          0.004734   

       number_diagnoses  gender_category  metformin_category  ...  \
0              0.889340        -0.017107           -0.001078  ...   
1              8.889340        -0.017107           -0.001078  ...   
2              6.889340         0.982893           -0.001078  ...   
3              4.889340         0.982893           -0.001078  ...   
4              8.889340         0.982893           -0.001078  ...   
...                 ...              ...                 ...  ...   
73445          8.889340        -0.017107           -0.001078  ...   
73446          9.344308         1.053227            1.003355  ...   
73447          9.344308         0.053227            0.003355  ...   
73448          8.889340        -0.017107           -0.001078  ...   
73449          8.889340         0.982893           -0.001078  ...   

       glipizide-metformin_category  glimepiride-pioglitazone_category  \
0                         -0.000026                          -0.000004   
1                         -0.000026                          -0.000004   
2                         -0.000026                          -0.000004   
3                         -0.000026                          -0.000004   
4                         -0.000026                          -0.000004   
...                             ...                                ...   
73445                     -0.000026                          -0.000004   
73446                      0.000082                           0.000014   
73447                      0.000082                           0.000014   
73448                     -0.000026                          -0.000004   
73449                     -0.000026                          -0.000004   

       metformin-rosiglitazone_category  metformin-pioglitazone_category  \
0                                   0.0                        -0.000004   
1                                   0.0                        -0.000004   
2                                   0.0                        -0.000004   
3                                   0.0                        -0.000004   
4                                   0.0                        -0.000004   
...                                 ...                              ...   
73445         

### Threshold Optimizer
Post-processing: change predictions, mitigated dataset is equal to the original one

SET THE MODEL TO CLASSIFIER - NO MODEL ENFORCEMENT

NO PARAMETERS

NO MITIGATED DATASET

In [12]:
print(fl_to(config))

  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:285: FutureWarning: 'predict_method' default value is changed from 'predict' to 'auto'. Explicitly pass `predict_method='predict' to replicate the old behavior, or pass `predict_method='auto' or other valid values to silence this warning.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/fairlearn/postprocessing/_threshold_optimizer.py:285: FutureWarning: 'predict_method' default value is changed fr

{
    'Logistic Regression': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 1])
        },
        2: {
            'y_test': array([1, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([1, 1, 1, ..., 0, 0, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        3: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 0])
        },
        4: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        5: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        6: {
            'y_test': array([1, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 0, 1, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        7: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        8: {
            'y_test': array([0, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 0, 1, 1])
        }
    },
    'Decision Tree': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 1, ..., 0, 1, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 1])
        },
        2: {
            'y_test': array([1, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        3: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 0])
        },
        4: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        5: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([1, 0, 0, ..., 0, 1, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        6: {
            'y_test': array([1, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 1, 1, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        7: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        8: {
            'y_test': array([0, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 1, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 0, 1, 1])
        }
    },
    'Bagging': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test

# AIF360
[AIF360](https://aif360.mybluemix.net) is a system used in order to detect and mitigate a possible bias inside a dataset.
It requires firstly the identification of the corresponding protected attribute (sex, race, age etc.)

## Pre-processing

### Reweighing

Pre-processing: dataset is equal to the original one, the predictor is the same, only use different weights

NO PARAMETER

NO MITIGATED DATASET

NO MODEL ENFORCEMENT

In [13]:
print(aif360_rw(config))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

{
    'Logistic Regression': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 1])
        },
        2: {
            'y_test': array([1, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([1, 1, 1, ..., 0, 0, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        3: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 0])
        },
        4: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        5: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        6: {
            'y_test': array([1, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 0, 1, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        7: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        8: {
            'y_test': array([0, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 0, 1, 1])
        }
    },
    'Decision Tree': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 1, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 1, ..., 0, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 1])
        },
        2: {
            'y_test': array([1, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        3: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 0])
        },
        4: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        5: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([1, 0, 0, ..., 0, 1, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        6: {
            'y_test': array([1, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        7: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 1, 1, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        8: {
            'y_test': array([0, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 1, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 1, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 0, 1, 1])
        }
    },
    'Bagging': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test

### Disparate Impact Remover

Pre-processing: It transforms the dataset in a new mitigated one

PARAMETER REPAIR LEVEL

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [14]:
print(aif360_di(config))

  0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

{
    'Logistic Regression': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 1])
        },
        2: {
            'y_test': array([1, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([1, 1, 1, ..., 0, 0, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        3: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 0])
        },
        4: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        5: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        6: {
            'y_test': array([1, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 0, 1, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        7: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        8: {
            'y_test': array([0, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 0, 1, 1])
        }
    },
    'Decision Tree': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 1, ..., 0, 1, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 1])
        },
        2: {
            'y_test': array([1, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        3: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 0])
        },
        4: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        5: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([1, 0, 0, ..., 0, 1, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        6: {
            'y_test': array([1, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 1, 1, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        7: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        8: {
            'y_test': array([0, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 1, 1, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 1, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 0, 1, 1])
        }
    },
    'Bagging': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test

### Learning Fair Representation

PARAMETERS k, Ax, Ay, Az maxitem e maxfun

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [18]:
print( aif360_lfr(config))

Output hidden; open in https://colab.research.google.com to view.

### Optimized Preprocessing

Pre-processing: mitigated dataset
Richiede valori binari come input quindi è stato fatto un pre-processing sui valori per essere 0,1

OptTool difficile da creare, chiesto a Daniel delucidazioni

PARAMETERS distortion_fun, epsilon, clist, dlist

MITIGATED DATASET

NO MODEL ENFORCEMENT

In [15]:
print(aif360_op(config))

Output hidden; open in https://colab.research.google.com to view.

##In-processing

Ogni tecnica genera un nuovo classificatore: no score and no mitigated dataset

### Adversarial Debiasing
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

PARAMETER SESSION

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT

In [16]:
print(aif360_ad(config))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 3.654990
epoch 0; iter: 200; batch classifier loss: 0.913605
epoch 0; iter: 400; batch classifier loss: 0.689701
epoch 1; iter: 0; batch classifier loss: 0.702546
epoch 1; iter: 200; batch classifier loss: 0.647359
epoch 1; iter: 400; batch classifier loss: 0.688360
epoch 2; iter: 0; batch classifier loss: 0.657379
epoch 2; iter: 200; batch classifier loss: 0.667781
epoch 2; iter: 400; batch classifier loss: 0.646862
epoch 3; iter: 0; batch classifier loss: 0.659899
epoch 3; iter: 200; batch classifier loss: 0.654502
epoch 3; iter: 400; batch classifier loss: 0.698842
epoch 4; iter: 0; batch classifier loss: 0.656443
epoch 4; iter: 200; batch classifier loss: 0.638435
epoch 4; iter: 400; batch classifier loss: 0.605260
epoch 5; iter: 0; batch classifier loss: 0.642529
epoch 5; iter: 200; batch classifier loss: 0.641642
epoch 5; iter: 400; batch classifier loss: 0.664297
epoch 6; iter: 0; batch classifier loss: 0.656113
epoch 6; iter: 200; batch 

{
    0: {
        'y_test': array([0, 1, 0, ..., 1, 0, 0]),
        'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    },
    1: {
        'y_test': array([0, 0, 0, ..., 1, 0, 0]),
        'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
        's_test': array([1, 1, 0, ..., 1, 0, 1])
    },
    2: {
        'y_test': array([1, 1, 0, ..., 0, 0, 0]),
        'y_pred': array([1, 1, 1, ..., 0, 0, 0]),
        's_test': array([0, 1, 1, ..., 0, 1, 1])
    },
    3: {
        'y_test': array([0, 0, 0, ..., 0, 0, 0]),
        'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
        's_test': array([0, 1, 1, ..., 0, 1, 0])
    },
    4: {
        'y_test': array([0, 0, 0, ..., 1, 0, 0]),
        'y_pred': array([0, 1, 0, ..., 1, 0, 0]),
        's_test': array([1, 1, 0, ..., 0, 1, 1])
    },
    5: {
        'y_test': array([1, 1, 0, ..., 1, 0, 0]),
        'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    },
    6: {
        'y_test': array([1, 1, 0, ..., 1, 1, 0]),
        'y_pred': array([0, 1, 1, ..., 1, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    },
    7: {
        'y_test': array([0, 1, 0, ..., 1, 0, 1]),
        'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
        's_test': array([0, 1, 1, ..., 0, 1, 1])
    },
    8: {
        'y_test': array([0, 0, 0, ..., 0, 0, 1]),
        'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
        's_test': array([0, 1, 1, ..., 1, 1, 1])
    },
    9: {
        'y_test': array([0, 1, 0, ..., 0, 0, 0]),
        'y_pred': array([1, 1, 1, ..., 1, 1, 0]),
        's_test': array([1, 1, 1, ..., 0, 1, 1])
    }
}

### Prejudice remover
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

NON FUNZIONA!

PARAMETER eta

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT



In [21]:
print(aif360_pr(config))

{
    0: {
        'y_test': array([0, 1, 0, ..., 1, 0, 0]),
        'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    },
    1: {
        'y_test': array([0, 0, 0, ..., 1, 0, 0]),
        'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
        's_test': array([1, 1, 0, ..., 1, 0, 1])
    },
    2: {
        'y_test': array([1, 1, 0, ..., 0, 0, 0]),
        'y_pred': array([1, 1, 1, ..., 0, 0, 0]),
        's_test': array([0, 1, 1, ..., 0, 1, 1])
    },
    3: {
        'y_test': array([0, 0, 0, ..., 0, 0, 0]),
        'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
        's_test': array([0, 1, 1, ..., 0, 1, 0])
    },
    4: {
        'y_test': array([0, 0, 0, ..., 1, 0, 0]),
        'y_pred': array([0, 1, 0, ..., 1, 0, 0]),
        's_test': array([1, 1, 0, ..., 0, 1, 1])
    },
    5: {
        'y_test': array([1, 1, 0, ..., 1, 0, 0]),
        'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    },
    6: {
        'y_test': array([1, 1, 0, ..., 1, 1, 0]),
        'y_pred': array([0, 0, 1, ..., 1, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    },
    7: {
        'y_test': array([0, 1, 0, ..., 1, 0, 1]),
        'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
        's_test': array([0, 1, 1, ..., 0, 1, 1])
    },
    8: {
        'y_test': array([0, 0, 0, ..., 0, 0, 1]),
        'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
        's_test': array([0, 1, 1, ..., 1, 1, 1])
    },
    9: {
        'y_test': array([0, 1, 0, ..., 0, 0, 0]),
        'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
        's_test': array([1, 1, 1, ..., 0, 1, 1])
    }
}

### Exponentiated Gradient Reduction
In-processing: dataset is equal to the original one, the predictor is fairer by construction and given

PARAMETERS constraint e estimator

NO MITIGATED DATASET

NO SCORE FUNCTION

MODEL ENFORCEMENT

In [22]:
print(aif360_er(config))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/fairlearn/reductions/_moments/utility_parity.py:185: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updatin

{
    0: {
        'y_test': array([0, 1, 0, ..., 1, 0, 0]),
        'y_pred': array([0, 1, 0, ..., 0, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    },
    1: {
        'y_test': array([0, 0, 0, ..., 1, 0, 0]),
        'y_pred': array([0, 0, 1, ..., 0, 0, 1]),
        's_test': array([1, 1, 0, ..., 1, 0, 1])
    },
    2: {
        'y_test': array([1, 1, 0, ..., 0, 0, 0]),
        'y_pred': array([0, 1, 1, ..., 0, 0, 1]),
        's_test': array([0, 1, 1, ..., 0, 1, 1])
    },
    3: {
        'y_test': array([0, 0, 0, ..., 0, 0, 0]),
        'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
        's_test': array([0, 1, 1, ..., 0, 1, 0])
    },
    4: {
        'y_test': array([0, 0, 0, ..., 1, 0, 0]),
        'y_pred': array([0, 0, 1, ..., 1, 0, 0]),
        's_test': array([1, 1, 0, ..., 0, 1, 1])
    },
    5: {
        'y_test': array([1, 1, 0, ..., 1, 0, 0]),
        'y_pred': array([1, 0, 0, ..., 0, 1, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    },
    6: {
        'y_test': array([1, 1, 0, ..., 1, 1, 0]),
        'y_pred': array([0, 1, 0, ..., 1, 0, 0]),
        's_test': array([1, 1, 1, ..., 1, 1, 1])
    },
    7: {
        'y_test': array([0, 1, 0, ..., 1, 0, 1]),
        'y_pred': array([0, 1, 0, ..., 1, 1, 1]),
        's_test': array([0, 1, 1, ..., 0, 1, 1])
    },
    8: {
        'y_test': array([0, 0, 0, ..., 0, 0, 1]),
        'y_pred': array([1, 0, 0, ..., 0, 1, 1]),
        's_test': array([0, 1, 1, ..., 1, 1, 1])
    },
    9: {
        'y_test': array([0, 1, 0, ..., 0, 0, 0]),
        'y_pred': array([0, 1, 1, ..., 1, 1, 0]),
        's_test': array([1, 1, 1, ..., 0, 1, 1])
    }
}

## Post-processing


### Calibrated Equalized Post-Processing

Post-processing: model's labels are changes, mitigated dataset is equal to the original one

PARAMETER cost_constraint

NO MITIGATED DATASET

NO EMBEDDED SCORE FUNCTION

NO MODEL ENFORCEMENT

In [23]:
print(aif360_ce(config))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

{
    'Logistic Regression': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 1])
        },
        2: {
            'y_test': array([1, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([1, 1, 1, ..., 0, 0, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        3: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 0])
        },
        4: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        5: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        6: {
            'y_test': array([1, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 0, 1, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        7: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        8: {
            'y_test': array([0, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
            's_test': array([1, 1, 1, ..., 0, 1, 1])
        }
    },
    'Decision Tree': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 1, ..., 0, 1, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 1])
        },
        2: {
            'y_test': array([1, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        3: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 0])
        },
        4: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        5: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([1, 1, 0, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        6: {
            'y_test': array([1, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 1, 1, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        7: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        8: {
            'y_test': array([0, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 1, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 1, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 0, 1, 1])
        }
    },
    'Bagging': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test

### Equalize Odds Post-Processing
Post-processing: model's labels are changes, mitigated dataset is equal to the original one

NO PARAMETER

NO MITIGATED DATASET

NO EMBEDDED SCORE FUNCTION

NO MODEL ENFORCEMENT

In [24]:
print(aif360_eo(config))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

{
    'Logistic Regression': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 1, 0, 1])
        },
        2: {
            'y_test': array([1, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([1, 1, 1, ..., 1, 0, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        3: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 0])
        },
        4: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        5: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        6: {
            'y_test': array([1, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 0, 1, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        7: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        8: {
            'y_test': array([0, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 0, 1, 1])
        }
    },
    'Decision Tree': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 1, 1]),
            's_test': array([1, 1, 0, ..., 1, 0, 1])
        },
        2: {
            'y_test': array([1, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        3: {
            'y_test': array([0, 0, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 1, 0]),
            's_test': array([0, 1, 1, ..., 0, 1, 0])
        },
        4: {
            'y_test': array([0, 0, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 1, 0, 0]),
            's_test': array([1, 1, 0, ..., 0, 1, 1])
        },
        5: {
            'y_test': array([1, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        6: {
            'y_test': array([1, 1, 0, ..., 1, 1, 0]),
            'y_pred': array([0, 1, 1, ..., 1, 0, 0]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        7: {
            'y_test': array([0, 1, 0, ..., 1, 0, 1]),
            'y_pred': array([0, 1, 0, ..., 0, 1, 1]),
            's_test': array([0, 1, 1, ..., 0, 1, 1])
        },
        8: {
            'y_test': array([0, 0, 0, ..., 0, 0, 1]),
            'y_pred': array([1, 0, 0, ..., 1, 1, 1]),
            's_test': array([0, 1, 1, ..., 1, 1, 1])
        },
        9: {
            'y_test': array([0, 1, 0, ..., 0, 0, 0]),
            'y_pred': array([0, 1, 0, ..., 1, 1, 0]),
            's_test': array([1, 1, 1, ..., 0, 1, 1])
        }
    },
    'Bagging': {
        0: {
            'y_test': array([0, 1, 0, ..., 1, 0, 0]),
            'y_pred': array([0, 0, 0, ..., 0, 0, 1]),
            's_test': array([1, 1, 1, ..., 1, 1, 1])
        },
        1: {
            'y_test

### Reject Option Classification

Post-processing: model's labels are changes, mitigated dataset is equal to the original one

NO MITIGATED DATASET

PARAMETERS: low_class_thresh,high_class_thresh, num_class_thresh, num_ROC_margin, metric_name, metric_ub, metric_lb

NO EMBEDDED SCORE FUCTION

NO MODEL ENFORCEMENT

In [25]:
print(aif360_roc(config))

/usr/local/lib/python3.10/dist-packages/aif360/algorithms/postprocessing/reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
/usr/local/lib/python3.10/dist-packages/aif360/algorithms/postprocessing/reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
/usr/local/lib/python3.10/dist-packages/aif360/algorithms/postprocessing/reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
/usr/local/lib/python3.10/dist-packages/aif360/algorithms/postprocessing/reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")
/usr/local/lib/python3.10/dist-packages/aif360/algorithms/postprocessing/reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  w

{'Logistic Regression': {0: {'y_test': array([0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
          1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
          0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1,
          1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
          0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0,
          1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0,
          1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1,
          1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
          0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1,
          1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
          1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
          1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
          0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 